In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qepy.calculator import QEpyCalculator
from qepy.io import QEInput

In [ ]:
from ase.io.trajectory import Trajectory
from ase.lattice.hexagonal import Graphene
from ase import Atoms
from ase.io import read,write
import py3Dmol

# Challenge solution

In [ ]:
from ase.build import bulk
atoms = bulk('Si','diamond',a=5.43)
write('si.xyz',atoms)
v = py3Dmol.view(data=open('./si.xyz','r').read(), style='sphere')
v.addUnitCell()
v.zoomTo()
v.show()

In [ ]:
qe_options = {
    '&control': {
        'calculation': "'scf'",
        'prefix': "'tmp'",
        'pseudo_dir': "'./'"},
    '&system': {
        'ibrav' : 0,
        'ecutwfc': 20,
        'ecutrho': 200},
    '&electrons': {
        'conv_thr' : 1.0e-8},
     'atomic_species': ['Si 28.0855 Si.pbe-n-rrkjus_psl.1.0.0.UPF'],
     'k_points automatic': [' 9 9 9 0 0 0'],
}

In [ ]:
additional_files = ['https://pseudopotentials.quantum-espresso.org/upf_files/Si.pbe-n-rrkjus_psl.1.0.0.UPF']
from dftpy.formats import download_files
download_files(additional_files)

In [ ]:
calc = QEpyCalculator(qe_options=qe_options, logfile='tmp.out')
atoms.calc=calc

In [ ]:
energy = atoms.get_potential_energy()
efermi = calc.get_fermi_level()
print(efermi)

In [ ]:
qe_options['k_points automatic']= ['12 12 12 0 0 0']
energies, dos =  atoms.calc.get_dos(qe_options, width=0.6)

In [ ]:
emin = -12.0
emax = 9.0
plt.plot(energies, dos)
plt.xlim(emin, emax)
plt.axvline(x=0, ls='--')
plt.xlabel('Energy (Ry)')
plt.ylabel('DOS')

In [ ]:
lat = atoms.cell.get_bravais_lattice()
lat.plot_bz(show=True)
print(lat.description())

In [ ]:
path = atoms.cell.bandpath('GXWKG',npoints=60)

In [ ]:
qe_options['&system']['nbnd']=12
band = atoms.calc.get_band_structure(qe_options, kpts=path, reference=efermi) 

In [ ]:
bp=band.plot()
bp.set_ylim(-12,16)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([.12, .07, .67, .85])

emin = -12.0
emax = 12

bands = band.subtract_reference()
bands.plot(ax=ax, emin=emin, emax=emax)

dosax = fig.add_axes([.8, .07, .17, .85])
dosax.plot(dos, energies)

dosax.set_ylim(emin, emax)
dosax.set_xlim(0, None)
dosax.set_yticks([])
dosax.set_xticks([])
dosax.set_xlabel("DOS", fontsize=18);